In [8]:
import os
from datasets import load_dataset

In [9]:
def coco_to_yolo(x, y, w, h, width, heigth):
    return [((2 * x + w) / (2 * width)), ((2 * y + h) / (2 * heigth)), w / width, h / heigth]

In [4]:
def preprocessing(partition: str, data :any):
    os.makedirs(f"datasets/images/{partition}", exist_ok=True)
    os.makedirs(f"datasets/labels/{partition}", exist_ok=True)

    # data["train"][0] 
    for i, sample in enumerate(data[partition]):
        image = sample['image']
        label = sample["objects"]['category']
        bboxes = sample["objects"]['bbox']
        width = sample['width']
        height = sample['height']

        # processing bboxes
        items = []
        for label, bbox in zip(label, bboxes):
            xc, yc, w, h = coco_to_yolo(bbox[0], bbox[1], bbox[2], bbox[1], width, height)
            items.append(f"{label} {xc} {yc} {w} {h}")
        
        with open(f"datasets/labels/{partition}/{i}.txt", "w") as f:
            for item in items:
                f.write(f"{item}\n")

        image.save(f"datasets/images/{partition}/{i}.jpg")

In [7]:
from roboflow import Roboflow
rf = Roboflow(api_key="EvbgCXrdqvuW3r4D9tkJ")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in yolov8:: 100%|██████████| 20940/20940 [01:15<00:00, 277.46it/s]


In [1]:
if (__name__ == "__main__"):
    # data = load_dataset("https://universe.roboflow.com/ds/OpmEyv9HEN?key=aT00D1ddg4", "full")

    preprocessing("train", dataset)
    preprocessing("test", dataset)
    preprocessing("validation", dataset)

NameError: name 'preprocessing' is not defined